In [45]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch import Tensor

In [46]:
class NCF_EWC(nn.Module):
    def __init__(self, num_users, num_items, emb_size=8):
        super(NCF_EWC, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.ewc_lambda = 0.5  # ewc 중요도 계수, hyper param
        self.prev_params = {}  # 이전 학습에서 계산된 중요도 정보를 저장

    def forward(self, user_indices, item_indices):
        user_emb = self.user_emb(user_indices)
        item_emb = self.item_emb(item_indices)
        concat = torch.cat((user_emb, item_emb), dim=1)
        output = self.fc_layers(concat)
        print(f"output: {output}")
        return output.squeeze()

    def update_ewc_penalty(self, criterion, prev_params):
        ewc_loss = 0
        for name, param in self.named_parameters():
            if name in prev_params:
                ewc_loss += torch.sum((param - prev_params[name]) ** 2) * self.ewc_lambda
        total_loss = criterion + ewc_loss
        return total_loss

    def train_model(self, train_loader, lr=0.001, epochs=10):
        criterion = nn.BCELoss()
        optimizer = Adam(self.parameters(), lr=lr)

        for epoch in range(epochs):
            for i, data in enumerate(train_loader):
                user_indices, item_indices, ratings = data
                user_indices = Tensor(user_indices)
                item_indices = Tensor(item_indices)
                ratings = Tensor(ratings.float())
                optimizer.zero_grad()
                outputs = self.forward(user_indices, item_indices)
                loss = criterion(outputs, ratings)
                if epoch > 0:
                    loss = self.update_ewc_penalty(loss, self.prev_params)
                loss.backward()
                optimizer.step()
            self.prev_params = {name: param.detach() for name, param in self.named_parameters()}

## 무비렌즈

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

### 데이터 준비

일단 태스크별로 단순히 8:2로 쪼갠다.

In [48]:
# 데이터 로드
dfs = []
for i in range(5):
    dfs.append(pd.read_csv(f"./dataset/Movielens/increase/ml_100k_inc{i}.csv"))

In [49]:
for df in dfs:
    print(df['item'].max())

1478
1446
1478
1469
1451


In [50]:
# 데이터 전처리
"""
필요한 컬럼은 유저, 아이템, rating

우선 간단하게 빨리 해보는게 중요하니,
rating이 5점이면 rating 컬럼을 1
아니라면 0로 바꾸자고.
"""
cols = ['user', 'item', 'rating']
for df in dfs:
    df['rating'] = df['rating'].apply(lambda x: 1 if x >= 5 else 0)

In [51]:
# train/test 8:2 분리
"""
이것도 사실, 조금 찜찜하지만 우선 랜덤 8:2
"""
random_seed = 42

train0, test0 = train_test_split(dfs[0], test_size=0.2, random_state=random_seed)
train1, test1 = train_test_split(dfs[1], test_size=0.2, random_state=random_seed)
train2, test2 = train_test_split(dfs[2], test_size=0.2, random_state=random_seed)
train3, test3 = train_test_split(dfs[3], test_size=0.2, random_state=random_seed)
train4, test4 = train_test_split(dfs[4], test_size=0.2, random_state=random_seed)

## dataloader 정의

In [52]:
class MovielensDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx]['user']
        item = self.df.iloc[idx]['item']
        rating = self.df.iloc[idx]['rating']
        return user, item, rating

## 모델 정의

In [53]:
# NCF 모델
class NCF(nn.Module):
    def __init__(self, n_users, n_movies, emb_size=8, hidden_size=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(n_users, emb_size)
        self.movie_embedding = nn.Embedding(n_movies, emb_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, user_input, movie_input):
        user_embedded = self.user_embedding(user_input)
        movie_embedded = self.movie_embedding(movie_input)
        input_concat = torch.cat([user_embedded, movie_embedded], dim=-1)
        prediction = self.fc_layers(input_concat)
        return prediction

In [54]:
# gpu 설정
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

## 모델 train/test 함수 정의

In [55]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [62]:
def recall_at_k(output, target, k):
    _, idx = torch.topk(output, k=k)
    hit = torch.sum(target[idx])
    return hit.float() / target.sum().float() if target.sum().float() else torch.Tensor([0])

def test(model, device, test_loader, k=20):
    model.eval()
    criterion = nn.BCELoss()

    test_loss = 0
    test_recall = 0
    with torch.no_grad():
        for user, item, rating in test_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            output = model(user, item).squeeze()
            loss = criterion(output, rating.float())
            test_loss += loss.item()
            test_recall += recall_at_k(output, rating, k).item() # recall@20 기준
    test_loss /= len(test_loader)
    test_recall /= len(test_loader)

    return test_loss, test_recall

### 모델 학습

1. Naive
2. EWC

1. Naive

In [63]:
# 모델 객체 생성
n_users = train0['user'].max()+1
n_movies = train0['item'].max()+1
model = NCF(n_users, n_movies).to(device)
# 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [64]:
# 데이터 로더 설정
train_dataset0 = MovielensDataset(train0)
test_dataset0 = MovielensDataset(test0)

train_loader0 = DataLoader(train_dataset0, batch_size=64, shuffle=True)
test_loader0 = DataLoader(test_dataset0, batch_size=64, shuffle=False)

In [65]:
# train
epoch = 3
for e in tqdm(range(1, epoch+1)):
    train(model, device, train_loader0, optimizer, e)

 33%|███▎      | 1/3 [00:09<00:19,  9.51s/it]

Train Epoch: 1 	Loss: 0.570234


 67%|██████▋   | 2/3 [00:18<00:09,  9.33s/it]

Train Epoch: 2 	Loss: 0.547015


100%|██████████| 3/3 [00:27<00:00,  9.28s/it]

Train Epoch: 3 	Loss: 0.521258


In [66]:
# test
loss, recall20 = test(model, device, test_loader0)
print(recall20)

0.4236058492890813


우선 모든 데이터에 대해 incremental training을 하고 test해보자

In [67]:
for i in range(len(dfs)-1):
    if i == 0:
        # base block train-test

        # 데이터 준비
        train0, test0 = train_test_split(dfs[0], test_size=0.2, shuffle=False)
        train_dataset0 = MovielensDataset(train0)
        test_dataset0 = MovielensDataset(test0)
        train_loader0 = DataLoader(train_dataset0, batch_size=64, shuffle=False)
        test_loader0 = DataLoader(test_dataset0, batch_size=64, shuffle=False)

        # 모델 객체 생성
        n_users = train0['user'].max()+1
        n_movies = train0['item'].max()+1
        model = NCF(n_users, n_movies).to(device)
        # 옵티마이저 설정
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # train
        epoch = 1
        print(f"************** Train Start At TASK{i}")
        for e in tqdm(range(1, epoch+1)):
            train(model, device, train_loader0, optimizer, e)

        # test
        _, recall20 = test(model, device, test_loader0)
        print(f"******* At {i} TASK recall20 = {recall20}\n")

    else:
        # inc block train-test

        # 데이터 준비
        train_dataset = MovielensDataset(dfs[i])
        test_dataset = MovielensDataset(dfs[i+1]) # inc 블록에서는 다음 inc 블록을 test로 사용
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # train
        epoch = 1
        print(f"************** Train Start At TASK{i}")
        for e in tqdm(range(1, epoch+1)):
            train(model, device, train_loader, optimizer, e)

        # test
        _, recall20 = test(model, device, test_loader)
        print(f"******* At {i} TASK recall20 = {recall20}\n")

************** Train Start At TASK0


100%|██████████| 1/1 [00:09<00:00,  9.37s/it]

Train Epoch: 1 	Loss: 0.574624


******* At 0 TASK recall20 = 0.3661147016523914

************** Train Start At TASK1


100%|██████████| 1/1 [00:01<00:00,  1.05s/it]

...

Train Epoch: 1 	Loss: 0.565318


******* At 2 TASK recall20 = 0.3789854539852393

************** Train Start At TASK3


100%|██████████| 1/1 [00:01<00:00,  1.04s/it]

Train Epoch: 1 	Loss: 0.595232


******* At 3 TASK recall20 = 0.4027429768129399



2. EWC

In [68]:
# 모델 객체 생성
n_users = train0['user'].max()+1
n_movies = train0['item'].max()+1
model = NCF(n_users, n_movies).to(device)
# 옵티마이저 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [69]:
# EWC에 필요한 변수
fisher_dict = {}
optpar_dict = {}
ewc_lambda = 0.4 # ewc 강도 조절.. 높을수록 이전 파라미터의 중요도가 높아짐

In [70]:
# Task가 끝날 때 마다 optpar와 fisher를 저장해주는 함수.
def on_task_update(model, device, train_loader, optimizer, task_id):
    model.train()
    criterion = nn.BCELoss()
    optimizer.zero_grad()

    # accumulating gradients
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()

    fisher_dict[task_id] = {}
    optpar_dict[task_id] = {}

    # gradients accumulated can be used to calculate fisher
    for name, param in model.named_parameters():
        fisher_dict[task_id][name] = param.grad.data.clone().pow(2) # 누적 grad 값
        optpar_dict[task_id][name] = param.data.clone() # 최적 grad 값

In [71]:
# EWC를 적용한 train 함수
def train_ewc(model, device, train_loader, optimizer, epoch, task_id):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        train_loss += loss.item()

        # EWC 적용 부분
        for task in range(task_id):
            for name, param in model.named_parameters():
                fisher = fisher_dict[task][name]
                optpar = optpar_dict[task][name]
                train_loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda

        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [72]:
for i in range(len(dfs)-1):
    if i == 0:
        # base block train-test

        # 데이터 준비
        train0, test0 = train_test_split(dfs[0], test_size=0.2, shuffle=False)
        train_dataset0 = MovielensDataset(train0)
        test_dataset0 = MovielensDataset(test0)
        train_loader0 = DataLoader(train_dataset0, batch_size=64, shuffle=False)
        test_loader0 = DataLoader(test_dataset0, batch_size=64, shuffle=False)

        # 모델 객체 생성
        n_users = train0['user'].max()+1
        n_movies = train0['item'].max()+1
        model = NCF(n_users, n_movies).to(device)
        # 옵티마이저 설정
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # train
        epoch = 1
        print(f"************** Train Start At TASK{i}")
        for e in tqdm(range(1, epoch+1)):
            train_ewc(model, device, train_loader0, optimizer, e, i)
        on_task_update(model, device, train_loader0, optimizer, i)

        # test
        _, recall20 = test(model, device, test_loader0)
        print(f"******* At {i} TASK recall20 = {recall20}\n")

    else:
        # inc block train-test

        # 데이터 준비
        train_dataset = MovielensDataset(dfs[i])
        test_dataset = MovielensDataset(dfs[i+1]) # inc 블록에서는 다음 inc 블록을 test로 사용
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # train
        epoch = 1
        print(f"************** Train Start At TASK{i}")
        for e in tqdm(range(1, epoch+1)):
            train_ewc(model, device, train_loader, optimizer, e, i)
        on_task_update(model, device, train_loader, optimizer, i)

        # test
        _, recall20 = test(model, device, test_loader)
        print(f"******* At {i} TASK recall20 = {recall20}\n")

************** Train Start At TASK0


100%|██████████| 1/1 [00:09<00:00,  9.18s/it]

Train Epoch: 1 	Loss: 0.575395


******* At 0 TASK recall20 = 0.34986656705256214

************** Train Start At TASK1


100%|██████████| 1/1 [00:01<00:00,  1.03s/it]

...

Train Epoch: 1 	Loss: 0.648375


******* At 2 TASK recall20 = 0.36362729652931813

************** Train Start At TASK3


100%|██████████| 1/1 [00:01<00:00,  1.16s/it]

Train Epoch: 1 	Loss: 0.712408


******* At 3 TASK recall20 = 0.37396391392930556

